In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [4]:
V_protocol = pd.read_csv('data/vaccine_with_protocol.csv')
print(V_protocol.shape)
V_protocol.head( )

(6694, 5)


,vacc_date,zipcode,count_given,dose_nbr,protocol
0,2020-12-12,37215.0,1.0,1.0,two-shot
1,2020-12-14,37205.0,1.0,2.0,two-shot
2,2020-12-14,37215.0,1.0,1.0,two-shot
3,2020-12-15,37221.0,1.0,1.0,two-shot
4,2020-12-16,37215.0,1.0,1.0,two-shot


In [5]:
V_protocol.tail()

,vacc_date,zipcode,count_given,dose_nbr,protocol
6689,2021-04-07,37228.0,9.0,1.0,two-shot
6690,2021-04-07,37228.0,2.0,2.0,two-shot
6691,2021-04-07,NaN,8.0,1.0,two-shot
6692,2021-04-07,NaN,1.0,1.0,one-shot
6693,2021-04-07,NaN,7.0,2.0,two-shot


In [6]:
C_count = pd.read_csv('data/COVID_CountByZipPerDate 03292021.csv')
print(C_count.shape)
C_count.head( )

(8832, 3)


,Specimen Collection Date,Zip,Cases
0,3/6/2020,37217.0,1
1,3/10/2020,37214.0,1
2,3/11/2020,37013.0,1
3,3/11/2020,37204.0,2
4,3/11/2020,37205.0,1


In [8]:
C_count.tail()

,Specimen Collection Date,Zip,Cases
8827,3/27/2021,37210.0,1
8828,3/27/2021,37211.0,1
8829,3/27/2021,37212.0,1
8830,3/27/2021,37214.0,2
8831,3/27/2021,37221.0,2


In [14]:
C_count.dtypes



Specimen Collection Date     object
Zip                         float64
Cases                         int64
dtype: object

In [15]:
V_protocol.dtypes

vacc_date       object
zipcode        float64
count_given    float64
dose_nbr       float64
protocol        object
dtype: object